In [ ]:
import matplotlib.pyplot as plt
from math import ceil, exp
from random import randint

# Neural Network class
class NeuroNetwork:
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + exp(-x)) #exp(x) - return exponential value x: e^x
        # x can be of int or float type
    
    @staticmethod
    def sigmoid_derivative(x):
        return NeuroNetwork.sigmoid(x) * (1-NeuroNetwork.sigmoid(x))
    
    
    def __init__(self, input_l_size, output_l_size, hidden_layers_count=1, learning_rate=0.5):
        self.activate_func = NeuroNetwork.sigmoid
        self.derivate_func = NeuroNetwork.sigmoid_derivative
        self.learning_rate = learning_rate

        self.selected_layer = None #pointer to the layer
        self.l_count = hidden_layers_count +2 #2 = 1 input +  1 output
        hidden_l_size = min(input_l_size * 2 - 1, ceil(
            input_l_size * 2 / 3 + output_l_size))
        # based on the size of the inputted and outputted word
        # (math.ceil() - round the number with the decimal to big int number)
        self.layers = [self.add_layer(i, input_l_size, output_l_size, hidden_l_size) 
                   for i in range (self.l_count)] #range of i = [0..l_count]
        self.selected_layer = None #clean the pointer


    def add_layer(self, i, in_size, out_size, hl_size):
        count = i + 1  #range of i = [0..l_count]
        if 1 < count < self.l_count:    #hidden
            self.selected_layer = Layer(hl_size, self.selected_layer, self)
            # create new layer on the base of layer with the pointer
            # and point to the created layer
            return self.selected_layer
        if count == 1:
            self.selected_layer = Layer(in_size, None, self)
            return self.selected_layer
        # else: count == l_count -> outpput 
        self.selected_layer = Layer(out_size, self.selected_layer, self)
        return self.selected_layer
        
    def train(self, dataset, iters=1000):
        print(f'\nTRAINING STARTED({iters} iterations)...')
        for _ in range(iters):
            self.train_once(dataset)
        print(f'\nTRINING COMPLETED!\n')
    
    def test(self, data, op_name):
        print('\nTesting Data:')
        for case in data:
            self.set_input_data(case)
            res = self.get_prediction()
            print(f'{case[0]} {op_name} {case[1]} ~ {res[0]}')

    def train_once(self, dataset):
        for case in dataset:
            datacase = {'in_data': case[0], 'res': case[1]}
            #Example: datacase = {'in_data': [0,0], 'res': 0}
            self.set_input_data(datacase['in_data'])
            curr_res= self.get_prediction()
            for i in range(len(curr_res)):
                self.layers[self.l_count - 1].neurons[i].set_error(
                    curr_res[i] - datacase['res']
                ) #self.layers[self.l_count - 1] = out layer

    def set_input_data(self, val_list):
        self.layers[0].set_input_data(val_list)

    def get_prediction(self):
        layers = self.layers
        output_layer = layers[len(layers) - 1]
        out_data = [neuron.get_value() for neuron in output_layer.neurons]
        return out_data

# Class for the layer of the NN
class Layer:
    def __init__(self, layer_size, prev_layer, parent_network):
        self.prev_layer = prev_layer
        self.network = parent_network
        self.neurons = [Neuron(self, prev_layer) for i in range(layer_size)]

    def set_input_data(self, val_list):
        for i in range(len(val_list)):
            self.neurons[i].set_value(val_list[i])


# class for the different Neurons in the layers
class Neuron:
    def __init__(self, layer : Layer, previous_layer: Layer):
        self.value = 0
        self._layer = layer
        self.inputs = [Input(prev_neuron, randint(0, 10) / 10) for prev_neuron in 
                             previous_layer.neurons] if previous_layer else [] # generator of the list + one line condition
# random.randint(0, 10) / 10  random number from 0.0 to 1.0

        # if previous_layer: #if prev layer empty, then false, else true
        #     self.inputs = [Input(prev_neuron, randint(0,10) / 10) for previous_neuron in previous_layer.neurons]]
        # else:
        #     self.inputs = []
        self.get_value()
    
    def set_value(self, val):
        self.value = val
    
   
    
    def get_value(self):
        network = self._layer.network
        if not self.is_no_inputs():
            self.set_value(network.activate_func(self.get_input_sum()))
        return self.value
    
    def is_no_inputs(self):
        return not self.inputs
    
    def get_input_sum(self):
        total_sum = sum(curr_input.prev_neuron.get_value() * curr_input.weight for curr_input in self.inputs)
        return total_sum
    
    def set_error(self, val):
        if self.is_no_inputs():
            return
        w_delta = val * self._layer.network.derivate_func(self.get_input_sum())
        for curr_input in self.inputs:
            curr_input.weight -= curr_input.prev_neuron.get_value() * w_delta * self._layer.network.learning_rate
            curr_input.prev_neuron.set_error(curr_input.weight * w_delta)
            
# class for the input data
class Input:
    def __init__(self, prev_neuron: Neuron, weight):
        self.prev_neuron = prev_neuron
        self.weight = weight



# new_nn = NeuroNetwork(2,1)
# dataset_or = [[[0,0], 0], [[0,1], 1], [[1,0], 1], [[1,1],1]]
# new_nn.train(dataset_or, 100000)
# test_data = [[0,0], [0,1], [1,0], [1,1]]
# new_nn.test(test_data, 'OR')

# new_nn_and = NeuroNetwork(2,1)
# dataset_and = [[[0,0], 0], [[0,1], 0],[[1,0],0],[[1,1],1]]
# test_data = [[0,0], [0,1], [1,0], [1,1]]
# new_nn_and.train(dataset_and, 100000)
# new_nn_and.test(test_data, 'AND')

new_nn_xor = NeuroNetwork(2,1)
dataset_xor = [[[0,0], 0], [[0,1], 1],[[1,0],1],[[1,1],0]]
test_data = [[0,0], [0,1], [1,0], [1,1]]
new_nn_xor.train(dataset_xor, 300000) #for 10k 0 xor 0 ~ 1,5% inaccuracy, for 30k 0 xor 0 ~ 0.81%
new_nn_xor.test(test_data, 'XOR')

#TODO add function to recognize the color product of to colors combined
#TODO need to convert pixelvalue from img to float, then get the result as float, and convert it to color to show



TRAINING STARTED(300000 iterations)...

TRINING COMPLETED!


Testing Data:
0 XOR 0 ~ 0.008136399116713846
0 XOR 1 ~ 0.9951198770556317
1 XOR 0 ~ 0.9951197104514218
1 XOR 1 ~ 0.0010837256815157243


In [ ]:
print(abs(10000/0.0140)/100)
print(abs(30000/0.0081)/100)

7142.857142857143
37037.03703703704
